This is roughly the same code as in the video demo [here](https://youtu.be/TsX9lpTnWIU).  The `Cleaner Denver news example.ipynb` file is a slightly cleaner version of this same file.

In [1]:
import requests as req

In [2]:
res = req.get('https://www.denvergov.org/content/denvergov/en/city-of-denver-home/news.html')

In [3]:
from bs4 import BeautifulSoup as bs

soup = bs(res.content)

In [4]:
divs = soup.find_all('div', {'class': 'denver-news-list-item'})

In [5]:
type(divs[0])

bs4.element.Tag

In [6]:
divs[0]

<div class="denver-news-list-item">
<p class="denver-news-list-date">Dec 2, 2019</p>
<h3><a href="/content/denvergov/en/environmental-health/about-us/news-room/newsroom_2019/fifth-annual-sustainable-denver-summit.html">City to Host Fifth Annual Sustainable Denver Summit</a></h3>
<p>Mayor Michael B. Hancock and Denver’s Office of Sustainability host the fifth annual Sustainable Denver Summit on Thursday, Dec. 5, at the Colorado Convention Center. The Summit is the community’s premier gathering of industry leaders and professionals working on today’s most important sustainability solutions. Their work will help Denver achieve its ambitious 2020 Sustainability Goals and other efforts to improve sustainability and climate action.</p>
<div style="clear:both;"></div>
</div>

In [7]:
import pandas as pd

In [8]:
pd.to_datetime(divs[0].find('p', {'class': 'denver-news-list-date'}).text)

Timestamp('2019-12-02 00:00:00')

In [9]:
BASE_URL = 'https://www.denvergov.org'

In [10]:
divs[0].find('a')

<a href="/content/denvergov/en/environmental-health/about-us/news-room/newsroom_2019/fifth-annual-sustainable-denver-summit.html">City to Host Fifth Annual Sustainable Denver Summit</a>

In [11]:
divs[0].find('a').text

'City to Host Fifth Annual Sustainable Denver Summit'

In [12]:
divs[0].find('a').get('href')

'/content/denvergov/en/environmental-health/about-us/news-room/newsroom_2019/fifth-annual-sustainable-denver-summit.html'

In [13]:
BASE_URL + divs[0].find('a').get('href')

'https://www.denvergov.org/content/denvergov/en/environmental-health/about-us/news-room/newsroom_2019/fifth-annual-sustainable-denver-summit.html'

In [14]:
for d in divs:
    date = pd.to_datetime(d.find('p', {'class': 'denver-news-list-date'}).text)
    link = d.find('a')
    href = BASE_URL + link.get('href')
    title = link.text

In [15]:
title

'Denver Promoted to “Silver Medal” Status in the CityHealth Annual Policy Assessment'

In [16]:
date

Timestamp('2019-11-25 00:00:00')

In [17]:
href

'https://www.denvergov.org/content/denvergov/en/environmental-health/about-us/news-room/newsroom_2019/city-health-denver-receives-silver-medal-2019.html'

In [18]:
from pymongo import MongoClient
from tqdm import tqdm_notebook

PAGE_URL = 'https://www.denvergov.org/content/denvergov/en/city-of-denver-home/news.html?page={}'

client = MongoClient()
db = client['news']
coll = db['denver']


for page in tqdm_notebook(range(1, 71)):        
    url = PAGE_URL.format(page)
    res = req.get(url)
    soup = bs(res.content)
    divs = soup.find_all('div', {'class': 'denver-news-list-item'})
    
    for d in divs:
        date = pd.to_datetime(d.find('p', {'class': 'denver-news-list-date'}).text)
        link = d.find('a')
        href = BASE_URL + link.get('href')
        title = link.text
        
        coll.insert_one({'date': date,
                        'link': href,
                        'title': title})

client.close()

In [19]:
from pprint import pprint

client = MongoClient()
db = client['news']
coll = db['denver']

pprint(coll.find_one())

{'_id': ObjectId('5de581023ae5ef8671ee264a'),
 'date': datetime.datetime(2019, 12, 2, 0, 0),
 'link': 'https://www.denvergov.org/content/denvergov/en/environmental-health/about-us/news-room/newsroom_2019/fifth-annual-sustainable-denver-summit.html',
 'title': 'City to Host Fifth Annual Sustainable Denver Summit'}


In [20]:
with open('test.html', 'wb') as f:
    f.write(res.content)